In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'brain-tumor-classification-mri:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F672377%2F1183165%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240501%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240501T165553Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D432cfe9f18d7277eb9586b329fc1c198d20daa18572a6e00f6646b43fd12af5f7323aad2cd2058265887a0922955ddd0e2943d361783efefbabacf24e3f52227558ff08c18ee0d4b4af5b491106750d0d9a6441ab2dceab03d389e0e2666c379ec904bb4f416638ec8c43a22a19a494cc214a45ea1a376d8cc9a713cf425f5cf30694f36f6756b3454a2ba36a1384c4c4675b6834f732f0b7a2af23d9f5f251a561ad5f86c7e66c1bdaa0b499bda6a25d605b51989742baee854c37deb4de80ec5ee9d997844c7ed88359ae0a477a98ebf41eacf612c291db973a3c04fba8d7e5f983be646c6b2d7afdbb5a222c635b597d7569796b7ebab1fb3f56e7dd02af6,brain-tumors-256x256:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3754858%2F6496514%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240501%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240501T165554Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2e5912f3e12f52d37cd102582aa5246298d12550391dc774d6d61cc5baa50fc0d1f3efbf6f1b49c3c0191da3e53ae77cbad7415889229c352d01f210d39ccd6286530dc5867a59ab22b867e5550a51a3a0728c4869e56213f4406767712a40133b2449aa252b55b1a9bdf96c54bc32d313a3b3757213c25109486e41d79db54d3c8303c135981e835a50709df688ec6b2a8dd4db0f01f689d3bd1b7a536ac6bc48c73e30707c2e848569c9d13be840f757a31b153aca27389c950e4d3fd4533361c0efb84f41581e5e4c8591bdae63fea4807a62daa1b604878e55c855e130b261ab65b82c5528038b3b5aa09f5bf7ce9fe273611ffc1866d01cb782356de3c7'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 91002358 bytes downloaded
Downloaded and uncompressed: brain-tumor-classification-mri
[==================================================] 62292733 bytes downloaded
Downloaded and uncompressed: brain-tumors-256x256
Data source import complete.


In [2]:
#Importing necessary modules
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
import  tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.layers import Dense
from functools import partial
from tensorflow import keras
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam , Adamax



In [3]:
#defining paths of directory
train_dir1="/kaggle/input/brain-tumor-classification-mri/Training"
image_size=(224,224)
#loading and preprocessing training data
train_ds1 = tf.keras.utils.image_dataset_from_directory(
    train_dir1,
    labels='inferred',
    label_mode='categorical',#creating categorical labels
    validation_split=0.1,#defining a validation split
    subset='training',
    seed=123,
    image_size=image_size,#used to resize images to the required image size
    batch_size=32,#creating images of batch size 32
    shuffle=True)

#defining validation datasets
validation_ds1 = tf.keras.utils.image_dataset_from_directory(
    train_dir1,
    labels='inferred',
    label_mode='categorical',
    validation_split=0.1,
    subset='validation',
    seed=123,
    image_size=image_size,
    batch_size=32,
    shuffle=True)

Found 2870 files belonging to 4 classes.
Using 2583 files for training.
Found 2870 files belonging to 4 classes.
Using 287 files for validation.


In [4]:

class ConcatLayer(Layer):#creating custom classes for concatenating layers
    def __init__(self, **kwargs):
        super(ConcatLayer, self).__init__(**kwargs)

    def call(self, inputs):
        return tf.concat(inputs, axis=3)
#defining inception module
def inception_module(input_layer,f1,f2_convol1,f2_convol3,f3_convol1,f3_convol5,f4):

    #input_layer:input
    # - f1: number of filters of the 1x1 convolutional layer in the first path
    # - f2_convol1, f2_convol3 are number of filters corresponding to the 1x1 and 3x3 convolutional layers in the second path
    # - f3_convol1, f3_convol5 are the number of filters corresponding to the 1x1 and 5x5  convolutional layer in the third path
    # - f4: number of filters of the 1x1 convolutional layer in the fourth path
    default_conv2d=partial(keras.layers.Conv2D,kernel_size=(3,3),activation='relu',padding='same')
    #1st_path
    path1=default_conv2d(filters=f1,kernel_size=(1,1))(input_layer)
    #2nd_path
    path2=default_conv2d(filters=f2_convol1,kernel_size=(1,1))(input_layer)
    path2=default_conv2d(filters=f2_convol1)(path2)
    #3rd_path
    path3=default_conv2d(filters=f3_convol1,kernel_size=(1,1))(input_layer)
    path3=default_conv2d(filters=f3_convol5,kernel_size=(5,5))(path3)
    #4th_path
    path4= MaxPooling2D((3,3), strides= (1,1), padding = 'same')(input_layer)
    path4=default_conv2d(filters=f4,kernel_size=1)(path4)
    #Output layer is a concatenation along the depth layer
    output_layer = ConcatLayer()([path1, path2, path3, path4])
    return output_layer

In [5]:
#defining custom class for local response normalisation
class local_response_Normalisation(Layer):
    def _init_(self, **kwargs):
        super(local_response_Normalisation, self)._init_(**kwargs)

    def call(self, M):
        return tf.nn.local_response_normalization(M)
#defining CNN architecture
def CNN_architecture():
    # input layer
    default_conv2d = partial(keras.layers.Conv2D, kernel_size=3, activation='selu', padding='same', strides=1)
    input_layer = Input(shape=(224, 224,3))

    # convolutional layer1: specifications(filters = 64, kernel_size = (7,7), strides = 2)
    M = default_conv2d(filters=64, kernel_size=7, strides=2, padding='valid')(input_layer)

    # max-pooling layer: specifications(pool size = (3,3), strides = 2)
    M = MaxPooling2D(pool_size=(3, 3), strides=2)(M)

    # applying local_response_normalization

    M=local_response_Normalisation()(M)

    # convolutional layer2: specifications(filters = 64, kernel_size = (1,1), strides = 1)
    M = default_conv2d(filters=64, kernel_size=1)(M)

    # convolutional layer3: specifications(filters = 192, kernel_size = (3,3), strides = 1)
    M = default_conv2d(filters=192)(M)

    # applying local_response_normalization
    M=local_response_Normalisation()(M)

    # 2nd max-pooling layer: specifications(pool size = (3,3), strides = 2)
    M = MaxPooling2D(pool_size=(3, 3), strides=2)(M)

    # 1st Inception block
    M = inception_module(M, f1=64, f2_convol1=96, f2_convol3=128, f3_convol1=16, f3_convol5=32, f4=32)

    # 2nd Inception block
    M = inception_module(M, f1=128, f2_convol1=128, f2_convol3=192, f3_convol1=32, f3_convol5=96, f4=64)

    # max-pooling layer: specifications( pool_size = (3,3), strides = 2)
    M = MaxPooling2D(pool_size=(3, 3), strides=2)(M)

    # 3rd Inception block
    M = inception_module(M, f1=192, f2_convol1=96, f2_convol3=208, f3_convol1=16, f3_convol5=48, f4=64)

    # 4th Inception block
    M = inception_module(M, f1=160, f2_convol1=112, f2_convol3=224, f3_convol1=24, f3_convol5=64, f4=64)

    # 5th Inception block
    M = inception_module(M, f1=128, f2_convol1=128, f2_convol3=256, f3_convol1=24, f3_convol5=64, f4=64)

    # 6th Inception block
    M = inception_module(M, f1=112, f2_convol1=144, f2_convol3=288, f3_convol1=32, f3_convol5=64, f4=64)

    # 7th Inception block
    M = inception_module(M, f1=256, f2_convol1=160, f2_convol3=320, f3_convol1=32, f3_convol5=128, f4=128)

    # max-pooling layer: specifications(pool_size = (3,3), strides = 2)
    M = MaxPooling2D(pool_size=(3, 3), strides=2)(M)

    # 8th Inception block
    M = inception_module(M, f1=256, f2_convol1=160, f2_convol3=320, f3_convol1=32, f3_convol5=128, f4=128)

    # 9th Inception block
    M = inception_module(M, f1=384, f2_convol1=192, f2_convol3=384, f3_convol1=48, f3_convol5=128, f4=128)

    # Global Average pooling layer
    M = GlobalAveragePooling2D(name='GAPL')(M)


    M = Dropout(0.4)(M)

    # output layer
    output = Dense(4, activation='softmax')(M)

    # model
    model = Model(input_layer, output, name='CNN_architecture')
    return model

model = CNN_architecture()
# model.summary()
model.summary()
#compiling model with the desired loss function and optimizer
model.compile(optimizer=Adamax(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


Model: "CNN_architecture"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 109, 109, 64)         9472      ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 54, 54, 64)           0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 local_response__normalisat  (None, 54, 54, 64)           0         ['max_pooling2d

In [13]:
#Early_Stopping used to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
#Fitting Model
history = model.fit(train_ds1,
                        epochs= 20,
                        validation_data = validation_ds1,
                        callbacks = early_stopping)

Epoch 1/20
81/81 [==============================] - 18s 217ms/step - loss: 0.2395 - accuracy: 0.9144 - val_loss: 0.3279 - val_accuracy: 0.8780
Epoch 2/20
81/81 [==============================] - 17s 207ms/step - loss: 0.2067 - accuracy: 0.9191 - val_loss: 0.3255 - val_accuracy: 0.8711
Epoch 3/20
81/81 [==============================] - 17s 208ms/step - loss: 0.2182 - accuracy: 0.9156 - val_loss: 0.3352 - val_accuracy: 0.8850
Epoch 4/20
81/81 [==============================] - 18s 214ms/step - loss: 0.2187 - accuracy: 0.9172 - val_loss: 0.3571 - val_accuracy: 0.8780
Epoch 5/20
81/81 [==============================] - 17s 211ms/step - loss: 0.2062 - accuracy: 0.9249 - val_loss: 0.2860 - val_accuracy: 0.8850
Epoch 6/20
81/81 [==============================] - 17s 205ms/step - loss: 0.1513 - accuracy: 0.9408 - val_loss: 0.3223 - val_accuracy: 0.9024
Epoch 7/20
81/81 [==============================] - 17s 211ms/step - loss: 0.2142 - accuracy: 0.9129 - val_loss: 0.3409 - val_accuracy: 0.8676

In [14]:
model.save("/content/brain_tumour/brain_tumour_trained.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
#defining directory for second dataset
train_dir2="/kaggle/input/brain-tumors-256x256/Data"
train_ds2 = tf.keras.utils.image_dataset_from_directory(
    train_dir2,
    labels='inferred',
    label_mode='categorical',#creating categorical labels
    validation_split=0.1,#defining a validation split
    subset='training',
    seed=123,
    image_size=image_size,#used to resize images to the required image size
    batch_size=32,#creating images of batch size 32
    shuffle=True)

#defining validation datasets
validation_ds2 = tf.keras.utils.image_dataset_from_directory(
    train_dir2,
    labels='inferred',
    label_mode='categorical',
    validation_split=0.1,
    subset='validation',
    seed=123,
    image_size=image_size,
    batch_size=32,
    shuffle=True)

Found 3096 files belonging to 4 classes.
Using 2787 files for training.
Found 3096 files belonging to 4 classes.
Using 309 files for validation.


In [18]:
import tensorflow as tf
import keras
#loading data to train on the second dataset

class LocalResponseNormalisation(tf.keras.layers.Layer):
        def _init_(self, **kwargs):
            super(local_response_Normalisation, self)._init_(**kwargs)

        def call(self, M):
            return tf.nn.local_response_normalization(M)
class ConcatLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(ConcatLayer, self).__init__(**kwargs)

    def call(self, inputs):
        return tf.concat(inputs, axis=3)

model = tf.keras.models.load_model("/content/brain_tumour/brain_tumour_trained.h5",custom_objects={"LocalResponseNormalisation": LocalResponseNormalisation,"ConcatLayer":ConcatLayer})


In [19]:
#Early_Stopping used to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
#Fitting Model
history = model.fit(train_ds2,
                        epochs= 20,
                        validation_data = validation_ds2,
                        callbacks = early_stopping)

Epoch 1/20
88/88 [==============================] - 36s 274ms/step - loss: 1.5156 - accuracy: 0.3003 - val_loss: 1.3139 - val_accuracy: 0.3074
Epoch 2/20
88/88 [==============================] - 18s 204ms/step - loss: 1.2253 - accuracy: 0.4331 - val_loss: 1.1383 - val_accuracy: 0.5243
Epoch 3/20
88/88 [==============================] - 18s 207ms/step - loss: 1.0947 - accuracy: 0.5379 - val_loss: 0.9811 - val_accuracy: 0.5469
Epoch 4/20
88/88 [==============================] - 18s 204ms/step - loss: 0.9146 - accuracy: 0.6118 - val_loss: 0.8942 - val_accuracy: 0.5761
Epoch 5/20
88/88 [==============================] - 19s 208ms/step - loss: 0.8192 - accuracy: 0.6656 - val_loss: 0.8328 - val_accuracy: 0.6408
Epoch 6/20
88/88 [==============================] - 18s 202ms/step - loss: 0.7083 - accuracy: 0.7043 - val_loss: 0.6893 - val_accuracy: 0.7184
Epoch 7/20
88/88 [==============================] - 18s 207ms/step - loss: 0.6425 - accuracy: 0.7280 - val_loss: 0.6173 - val_accuracy: 0.7443

In [20]:
model.save("/content/brain_tumour/brain_tumour_fullytrained.h5")#saving the updated model for future use